# Set-up

In [1]:
# load packages
import requests
from bs4 import BeautifulSoup

In [2]:
# Define the URL of the site
base_site = "https://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/2/"

In [3]:
# sending a request to the webpage
response = requests.get(base_site)
response.status_code

200

In [4]:
# get the HTML from the webpage
html = response.content

## Choosing a parser

### html.parser

In [5]:
# convert the HTML to a Beautiful Soup object
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Exporting the HTML to a file
with open('Rotten_tomatoes_page_2_HTML_Parser.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

In [7]:
# When inspecting the file we see that HTML element is closed at the begining -- it parsed incorrectly!
# Let's check another parser

### lxml

In [8]:
# convert the HTML to a BeatifulSoup object
soup = BeautifulSoup(html, 'lxml')

In [9]:
# Exporting the HTML to a file
with open('Rotten_tomatoes_page_2_LXML_Parser.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

In [10]:
# By first accounts of inspecting the file everything seems fine

### A word of caution

In [11]:
# Beautiful Soup ranks the lxml parser as the best one.

# If a parser is not explicitly stated in the Beautiful Soup constructor,
# the best one available on the current machine is chosen.

# This means that the same piece of code can give different results on different computers.

## Finding an element containing all the data

In [12]:
# Find all div tags on the webpage containing the information we want to scrape
divs = soup.find_all("div", {"class": "col-sm-18 col-full-xs countdown-item-content"})
divs

[<div class="col-sm-18 col-full-xs countdown-item-content">
 <div class="row countdown-item-title-bar">
 <div class="col-sm-20 col-full-xs" style="height: 100%;">
 <div class="article_movie_title" style="float: left;">
 <div><h2><a href="https://www.rottentomatoes.com/m/13_assassins_2011/">13 Assassins</a> <span class="subtle start-year">(2011)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">95%</span></h2></div>
 </div>
 </div>
 <div class="col-sm-4 col-full-xs" style="height: 100%;">
 <div class="countdown-index">#70</div>
 </div>
 </div>
 <div class="row countdown-item-details">
 <div class="col-sm-24">
 <div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>98.424% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placeme

# Extracting the title, year and score of each movie

In [13]:
# The title, year and score of each movie are contained in the 'h2' tags

In [14]:
# for instance, let's explore the first div
divs[0].find("h2")

<h2><a href="https://www.rottentomatoes.com/m/13_assassins_2011/">13 Assassins</a> <span class="subtle start-year">(2011)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">95%</span></h2>

In [15]:
# Extracting all 'h2' tags
headings = [div.find("h2") for div in divs]
headings

[<h2><a href="https://www.rottentomatoes.com/m/13_assassins_2011/">13 Assassins</a> <span class="subtle start-year">(2011)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">95%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/full_contact/">Full Contact</a> <span class="subtle start-year">(1992)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">88%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/indiana_jones_and_the_last_crusade/">Indiana Jones and the Last Crusade</a> <span class="subtle start-year">(1989)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">88%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/kung_fu_hustle/">Kung Fu Hustle</a> <span class="subtle start-year">(2005)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">90%</span></h2>,
 <h2><a href="https://www

In [16]:
# Inspecting the text inside the headings
[heading.text for heading in headings]

['13 Assassins (2011)  95%',
 'Full Contact (1992)  88%',
 'Indiana Jones and the Last Crusade (1989)  88%',
 'Kung Fu Hustle (2005)  90%',
 'A Better Tomorrow (2010)  93%',
 'Iron Man (2008)  94%',
 'The Night Comes For Us (2018)  90%',
 'Logan (2017)  93%',
 'Goldfinger (1964)  97%',
 'Assault on Precinct 13 (1976)  98%',
 'Wonder Woman (2017)  93%',
 'Fist of Fury (Jing wu men) (1972)  92%',
 'Captain America: The Winter Soldier (2014)  90%',
 'Oldboy (2005)  82%',
 'The French Connection (1971)  98%',
 'Furious 7 (2015)  81%',
 'La Femme Nikita (Nikita) (1990)  88%',
 'Supercop (1996)  96%',
 'Dirty Harry (1971)  91%',
 'Live Die Repeat: Edge of Tomorrow (2014)  90%',
 'X2: X-Men United (2003)  85%',
 'The Fugitive (1993)  96%',
 'Black Panther (2018)  97%',
 'Inception (2010)  87%',
 'Braveheart (1995)  77%',
 'Minority Report (2002)  90%',
 'Avengers: Endgame (2019)  94%',
 'Dredd (2012)  79%',
 'The Bourne Identity (2002)  83%',
 'Ip Man (2010)  85%',
 'Face/Off (1997)  92%',
 '

In [17]:
# It does contain the info we want to extract
# However, we need to obtain the title, year and score separately
# Let's inspect one heading to see if there is a way to distinguish between them
headings[0]

<h2><a href="https://www.rottentomatoes.com/m/13_assassins_2011/">13 Assassins</a> <span class="subtle start-year">(2011)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">95%</span></h2>

In [18]:
# We notice that:

# The movie title is in the 'a' tag
# The year is in a 'span' with class 'start-year'
# The score is in a 'span' with class 'tMeterScore'

## Title

In [19]:
# Let's check all heading links
[heading.find('a') for heading in headings]

[<a href="https://www.rottentomatoes.com/m/13_assassins_2011/">13 Assassins</a>,
 <a href="https://www.rottentomatoes.com/m/full_contact/">Full Contact</a>,
 <a href="https://www.rottentomatoes.com/m/indiana_jones_and_the_last_crusade/">Indiana Jones and the Last Crusade</a>,
 <a href="https://www.rottentomatoes.com/m/kung_fu_hustle/">Kung Fu Hustle</a>,
 <a href="https://www.rottentomatoes.com/m/better_tomorrow/">A Better Tomorrow</a>,
 <a href="https://www.rottentomatoes.com/m/iron_man/">Iron Man</a>,
 <a href="https://www.rottentomatoes.com/m/the_night_comes_for_us/">The Night Comes For Us</a>,
 <a href="https://www.rottentomatoes.com/m/logan_2017/">Logan</a>,
 <a href="https://www.rottentomatoes.com/m/goldfinger/">Goldfinger</a>,
 <a href="https://www.rottentomatoes.com/m/1001280-assault_on_precinct_13/">Assault on Precinct 13</a>,
 <a href="https://www.rottentomatoes.com/m/wonder_woman_2017/">Wonder Woman</a>,
 <a href="https://www.rottentomatoes.com/m/chinese_connection/">Fist of

In [20]:
# Obtaining the movie titles from the links
movie_names = [heading.find('a').string for heading in headings]
movie_names

['13 Assassins',
 'Full Contact',
 'Indiana Jones and the Last Crusade',
 'Kung Fu Hustle',
 'A Better Tomorrow',
 'Iron Man',
 'The Night Comes For Us',
 'Logan',
 'Goldfinger',
 'Assault on Precinct 13',
 'Wonder Woman',
 'Fist of Fury (Jing wu men)',
 'Captain America: The Winter Soldier',
 'Oldboy',
 'The French Connection',
 'Furious 7',
 'La Femme Nikita (Nikita)',
 'Supercop',
 'Dirty Harry',
 'Live Die Repeat: Edge of Tomorrow',
 'X2: X-Men United',
 'The Fugitive',
 'Black Panther',
 'Inception',
 'Braveheart',
 'Minority Report',
 'Avengers: Endgame',
 'Dredd',
 'The Bourne Identity',
 'Ip Man',
 'Face/Off',
 'To Live and Die in L.A.',
 'The Dark Knight',
 'Mission: Impossible Ghost Protocol',
 'Fast Five',
 'Lethal Weapon',
 'The Rock',
 'RoboCop',
 'John Wick: Chapter 2',
 'Casino Royale',
 'Baby Driver',
 'Fist of Legend (Jing wu ying xiong)',
 'The Killer',
 'The Raid 2',
 'Enter the Dragon',
 'Commando',
 'First Blood',
 'Mission: Impossible Rogue Nation',
 'The Terminat

## Year

In [21]:
# Filtering only the spans containing the year
[heading.find("span", class_ = 'start-year') for heading in headings]

[<span class="subtle start-year">(2011)</span>,
 <span class="subtle start-year">(1992)</span>,
 <span class="subtle start-year">(1989)</span>,
 <span class="subtle start-year">(2005)</span>,
 <span class="subtle start-year">(2010)</span>,
 <span class="subtle start-year">(2008)</span>,
 <span class="subtle start-year">(2018)</span>,
 <span class="subtle start-year">(2017)</span>,
 <span class="subtle start-year">(1964)</span>,
 <span class="subtle start-year">(1976)</span>,
 <span class="subtle start-year">(2017)</span>,
 <span class="subtle start-year">(1972)</span>,
 <span class="subtle start-year">(2014)</span>,
 <span class="subtle start-year">(2005)</span>,
 <span class="subtle start-year">(1971)</span>,
 <span class="subtle start-year">(2015)</span>,
 <span class="subtle start-year">(1990)</span>,
 <span class="subtle start-year">(1996)</span>,
 <span class="subtle start-year">(1971)</span>,
 <span class="subtle start-year">(2014)</span>,
 <span class="subtle start-year">(2003)<

In [22]:
# Extracting the year string
years = [heading.find("span", class_ = 'start-year').string for heading in headings]
years

['(2011)',
 '(1992)',
 '(1989)',
 '(2005)',
 '(2010)',
 '(2008)',
 '(2018)',
 '(2017)',
 '(1964)',
 '(1976)',
 '(2017)',
 '(1972)',
 '(2014)',
 '(2005)',
 '(1971)',
 '(2015)',
 '(1990)',
 '(1996)',
 '(1971)',
 '(2014)',
 '(2003)',
 '(1993)',
 '(2018)',
 '(2010)',
 '(1995)',
 '(2002)',
 '(2019)',
 '(2012)',
 '(2002)',
 '(2010)',
 '(1997)',
 '(1985)',
 '(2008)',
 '(2011)',
 '(2011)',
 '(1987)',
 '(1996)',
 '(1987)',
 '(2017)',
 '(2006)',
 '(2017)',
 '(1994)',
 '(1989)',
 '(2014)',
 '(1973)',
 '(1985)',
 '(1982)',
 '(2015)',
 '(1984)',
 '(2000)',
 '(2003)',
 '(1994)',
 '(1994)',
 '(1994)',
 '(2014)',
 '(2001)',
 '(1987)',
 '(2007)',
 '(1990)',
 '(1982)',
 '(1995)',
 '(2012)',
 '(2018)',
 '(1981)',
 '(1986)',
 '(1992)',
 '(1999)',
 '(1991)',
 '(1988)',
 '(2015)']

In [23]:
years[0]

'(2011)'

### Removing the brackets

In [24]:
# One way to remove the brackets is to drop the first and last symbol of the string
years[0][1:-1]

'2011'

In [25]:
# However, this will break, if the format of the year is changed

In [26]:
# Alternativelly, we can do it with the help of the strip() method (this is robust)

# It removes leading and trailing symbols from a string
# By default, it removes whitespace, but we can specify other symbols to strip

In [27]:
# Removing '('
years[0].strip('(')

'2011)'

In [28]:
# Removing ')'
years[0].strip(')')

'(2011'

In [29]:
# Combining both
years[0].strip('()')

'2011'

In [30]:
# Updating years with stripped values
years = [year.strip('()') for year in years]
years

['2011',
 '1992',
 '1989',
 '2005',
 '2010',
 '2008',
 '2018',
 '2017',
 '1964',
 '1976',
 '2017',
 '1972',
 '2014',
 '2005',
 '1971',
 '2015',
 '1990',
 '1996',
 '1971',
 '2014',
 '2003',
 '1993',
 '2018',
 '2010',
 '1995',
 '2002',
 '2019',
 '2012',
 '2002',
 '2010',
 '1997',
 '1985',
 '2008',
 '2011',
 '2011',
 '1987',
 '1996',
 '1987',
 '2017',
 '2006',
 '2017',
 '1994',
 '1989',
 '2014',
 '1973',
 '1985',
 '1982',
 '2015',
 '1984',
 '2000',
 '2003',
 '1994',
 '1994',
 '1994',
 '2014',
 '2001',
 '1987',
 '2007',
 '1990',
 '1982',
 '1995',
 '2012',
 '2018',
 '1981',
 '1986',
 '1992',
 '1999',
 '1991',
 '1988',
 '2015']

In [31]:
# Converting all the strings to integers
years = [int(year) for year in years]
years

[2011,
 1992,
 1989,
 2005,
 2010,
 2008,
 2018,
 2017,
 1964,
 1976,
 2017,
 1972,
 2014,
 2005,
 1971,
 2015,
 1990,
 1996,
 1971,
 2014,
 2003,
 1993,
 2018,
 2010,
 1995,
 2002,
 2019,
 2012,
 2002,
 2010,
 1997,
 1985,
 2008,
 2011,
 2011,
 1987,
 1996,
 1987,
 2017,
 2006,
 2017,
 1994,
 1989,
 2014,
 1973,
 1985,
 1982,
 2015,
 1984,
 2000,
 2003,
 1994,
 1994,
 1994,
 2014,
 2001,
 1987,
 2007,
 1990,
 1982,
 1995,
 2012,
 2018,
 1981,
 1986,
 1992,
 1999,
 1991,
 1988,
 2015]